# OpenAI Embeddings - 텍스트를 벡터로 변환하는 기술

## 개요

**문서 임베딩(Document Embedding)** 은 텍스트를 컴퓨터가 처리할 수 있는 **수치형 벡터** 로 변환하는 자연언어 처리 기술입니다.

### 임베딩의 핵심 개념

임베딩은 다음과 같은 변환 과정을 거칩니다:

- **텍스트**: 자연어로 작성된 문장이나 문서
- **벡터**: 고차원 공간의 수치 배열
- **AI 모델**: 의미론적 관계를 학습한 변환 모델

컴퓨터는 숫자 연산을 기반으로 동작하므로, 텍스트의 의미를 이해하고 처리하기 위해서는 수치 형태로의 변환이 필수적입니다.

### OpenAI Embeddings의 특징

| 특징 | 설명 |
|------|------|
| **의미적 이해** | 단순한 키워드 매칭이 아닌 문맥적 의미 파악 |
| **고성능** | GPT 모델 기반의 우수한 벡터 생성 품질 |
| **효율성** | 다양한 차원 옵션으로 용도별 최적화 가능 |

### 학습 내용

1. **환경 설정** - OpenAI API 구성
2. **모델 선택** - 용도별 적합한 모델 결정
3. **쿼리 임베딩** - 단일 텍스트의 벡터 변환
4. **문서 임베딩** - 다중 문서의 일괄 처리
5. **차원 조정** - 벡터 크기 최적화
6. **유사도 계산** - 텍스트 간 의미적 유사성 측정

### 실용적 활용 사례

| 분야 | 활용 예시 |
|------|----------|
| **검색 시스템** | 사용자 쿼리와 관련 문서 매칭 |
| **대화형 AI** | 사용자 질문 의도 분석 및 적절한 응답 생성 |
| **문서 분류** | 이메일 스팸 필터링, 뉴스 카테고리 자동 분류 |
| **추천 시스템** | 콘텐츠 간 유사성 기반 개인화 추천 |

참조: [OpenAI 임베딩 공식 문서](https://platform.openai.com/docs/guides/embeddings/embedding-models)

---

# 1. 환경 설정

OpenAI Embeddings를 사용하기 위한 **필수 구성 요소**를 설정합니다. API 키 설정과 추적 시스템 활성화가 필요합니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

# 2. 모델 정보

OpenAI에서 제공하는 **3가지 임베딩 모델**을 비교하여 적합한 모델을 선택할 수 있습니다. 각 모델은 **성능**, **비용**, **용도**에서 차이를 보입니다.

## 모델 비교

| 모델명 | 페이지당 비용 | MTEB 성능 점수 | 최대 입력 길이 | 추천 용도 |
|--------|-------------|-------------|------------|----------|
| **text-embedding-3-small** | 62,500 | 62.3% | 8,191 | **일반적 용도** (최고 가성비) |
| **text-embedding-3-large** | 9,615 | 64.6% | 8,191 | **고정밀 작업** (최고 성능) |
| **text-embedding-ada-002** | 12,500 | 61.0% | 8,191 | **레거시 지원** (구버전 호환성) |

## 모델 선택 기준

### text-embedding-3-small (본 튜토리얼 사용 모델)

- **장점**: 우수한 가성비, 빠른 처리 속도
- **적합한 상황**: 일반적인 문서 검색, 챗봇 응답, 추천 시스템
- **비용**: 가장 경제적인 선택

### text-embedding-3-large

- **장점**: 최고 성능, 미세한 의미 차이 감지
- **적합한 상황**: 법률 문서, 의료 텍스트 등 정밀도가 중요한 경우
- **비용**: 가장 높음

### text-embedding-ada-002

- **용도**: 기존 시스템의 호환성 유지
- **주의사항**: 신규 프로젝트에서는 3세대 모델 사용 권장

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩 객체를 생성합니다.
# 가성비가 우수하고 일반적인 용도에 적합한 모델입니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# 임베딩 테스트를 위한 한국어 샘플 문장을 정의합니다.
# 이 문장이 수치 벡터로 변환될 예정입니다.
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

# 3. 쿼리 임베딩

**쿼리 임베딩**은 **단일 텍스트**를 벡터로 변환하는 기본적인 임베딩 방법입니다.

## embed_query() 메소드

`embeddings.embed_query(text)` 함수는 **단일 텍스트를 1536차원의 벡터로 변환**하는 핵심 함수입니다.

### 동작 방식

| 단계 | 설명 | 예시 |
|------|------|------|
| **입력** | 텍스트 문장 | "안녕하세요" |
| **처리** | OpenAI API를 통한 의미 분석 | 자연어 처리 모델 실행 |
| **출력** | 1536개 숫자 배열 | [0.123, -0.456, ...] |

### 주요 활용 사례

- **검색 시스템**: 사용자 질문과 가장 유사한 문서 찾기
- **텍스트 분석**: 텍스트의 감정, 주제 등 특성 분석
- **대화 시스템**: 사용자 의도 파악하기

In [ ]:
# 텍스트를 OpenAI API를 통해 임베딩 벡터로 변환합니다.
# 결과는 1536개의 부동소수점 숫자로 구성된 리스트가 됩니다.
query_result = embeddings.embed_query(text)

## 결과 확인

생성된 임베딩 벡터의 **처음 5개 값**을 확인해봅시다.

실제 벡터는 1536개의 부동소수점 숫자로 구성되어 있지만, 전체 내용이 길어서 일부만 미리보기로 확인합니다.

In [ ]:
# 임베딩 벡터의 처음 5개 요소를 슬라이싱으로 확인합니다.
# 실제로는 1536개의 부동소수점 값이 들어있습니다.
query_result[:5]

# 4. 문서 임베딩

**문서 임베딩**은 **여러 개의 텍스트를 동시에** 벡터로 변환하는 효율적인 처리 방법입니다.

## embed_documents() 메소드

`embeddings.embed_documents()` 함수는 **여러 문서를 일괄 처리**하는 기능을 제공합니다.

### 쿼리 vs 문서 임베딩 비교

| 구분 | 쿼리 임베딩 | 문서 임베딩 |
|------|-------------|-------------|
| **입력** | 단일 텍스트 | 텍스트 리스트 |
| **출력** | 단일 벡터 | 벡터 리스트 |
| **용도** | 검색 쿼리, 사용자 질문 | 문서 컬렉션, 데이터베이스 구축 |

### 일괄 처리의 장점

| 장점 | 설명 |
|------|------|
| **처리 속도** | 한 번에 여러 문서 처리로 시간 절약 |
| **비용 효율성** | API 호출 횟수 감소로 비용 절약 |
| **안정성** | 네트워크 오류 위험 감소 |

In [ ]:
# 동일한 텍스트를 4개 포함한 리스트를 일괄 임베딩 처리합니다.
# 결과는 4개의 벡터로 구성된 리스트가 됩니다.
doc_result = embeddings.embed_documents([text, text, text, text])

## 결과 분석

문서 임베딩의 **구조**를 확인해봅시다.

- `doc_result`는 **4개의 벡터를 담은 리스트**입니다
- 각 벡터는 **1536개의 숫자**로 구성됩니다

In [ ]:
# 문서 임베딩 결과의 개수를 확인합니다 (4개 문서 = 4개 벡터).
len(doc_result)

In [ ]:
# 첫 번째 문서의 임베딩 벡터에서 처음 5개 값을 확인합니다.
# [0]은 첫 번째 문서, [:5]는 처음 5개 요소를 의미합니다.
doc_result[0][:5]

# 5. 차원 조정

`text-embedding-3` 시리즈 모델의 **특별한 기능** 중 하나는 **벡터 차원을 자유롭게 조정**할 수 있다는 것입니다.

## 차원 조정의 필요성

### 개념 이해

임베딩 벡터를 **아파트 층수**에 비유하면 다음과 같습니다:

- **1536차원**: 초고층 아파트 (많은 정보, 많은 저장공간 필요)
- **1024차원**: 중층 아파트 (적당한 정보, 적당한 공간)
- **512차원**: 저층 아파트 (기본 정보, 공간 절약)

### 차원 축소의 장점

| 차원 수 | 정확도 | 처리 속도 | 저장공간 | 추천 용도 |
|--------|-------|----------|---------|----------|
| **1536** | 최고 | 보통 | 많음 | 고정밀 작업 |
| **1024** | 높음 | 빠름 | 보통 | **일반적 용도** |
| **512** | 보통 | 매우 빠름 | 적음 | 실시간 처리 |

### 차원 축소 시나리오

- **비용 절약**: 저장 공간과 계산 시간 감소
- **속도 향상**: 벡터 연산 성능 개선
- **메모리 최적화**: 제한된 환경에서 효율적 운용

기본적으로 **text-embedding-3-small**은 **1536차원**의 벡터를 생성합니다.

In [ ]:
# 첫 번째 문서 벡터의 차원 수를 확인합니다.
# text-embedding-3-small 모델의 기본 차원은 1536입니다.
len(doc_result[0])

## 차원(dimensions) 매개변수 설정

`dimensions` 매개변수를 사용하면 벡터 크기를 원하는 값으로 조정할 수 있습니다.

### 성능 vs 효율성 트레이드오프

차원 수에 따른 특성을 비교하면 다음과 같습니다:

| 차원 수 | 정확도 | 속도 | 저장공간 | 추천 상황 |
|--------|-------|------|---------|----------|
| **1536** | 최고 | 보통 | 많음 | 고정밀도가 필요한 전문 작업 |
| **1024** | 높음 | 빠름 | 보통 | **대부분의 일반적 용도** |
| **512** | 보통 | 매우 빠름 | 적음 | 실시간 처리가 필요한 경우 |

### 권장 사항

- **일반적인 프로젝트**: 1024차원 (가성비 최적)
- **고정밀 분석**: 1536차원 (최대 성능)
- **대용량 처리**: 512차원 (속도 우선)

In [ ]:
# 차원 수를 1024로 조정한 임베딩 객체를 생성합니다.
# 기본 1536차원 대비 약 33% 메모리 절약 효과가 있습니다.
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [ ]:
# 1024차원으로 설정된 임베딩의 실제 벡터 길이를 확인합니다.
# 결과는 1024가 나와야 합니다.
len(embeddings_1024.embed_documents([text])[0])

# 6. 유사도 계산

임베딩의 **핵심 가치**는 텍스트 간의 **의미적 유사성**을 수치로 측정할 수 있다는 점입니다.

In [ ]:
# 유사도 계산을 위한 다양한 테스트 문장들을 정의합니다.
# 의미적 유사성을 비교하기 위해 한국어와 영어 문장을 혼합했습니다.
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [ ]:
# 코사인 유사도 계산을 위해 scikit-learn 라이브러리를 사용합니다.
from sklearn.metrics.pairwise import cosine_similarity

# 모든 테스트 문장들을 리스트로 묶어서 관리합니다.
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]

# 5개 문장을 모두 1024차원 벡터로 변환합니다 (일괄 처리).
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [ ]:
# 두 벡터 간의 코사인 유사도를 계산하는 헬퍼 함수입니다.
# 결과는 -1(완전 반대) ~ 1(완전 동일) 사이의 값으로 나타납니다.
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [ ]:
# 모든 문장 쌍에 대해 유사도를 계산하고 결과를 출력합니다.
# 중복을 피하기 위해 i < j 조건을 사용합니다.
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:  # 중복 비교를 피하기 위한 조건
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

## 유사도 결과 해석

위 결과를 통해 OpenAI Embeddings가 텍스트의 의미를 얼마나 **정확하게** 파악하는지 확인할 수 있습니다.

### 예상 결과 패턴

**유사도 점수** (0~1 범위, 1에 가까울수록 유사):

| 유사도 범위 | 비교 대상 | 특징 |
|-----------|----------|------|
| **0.99 이상** | `"안녕하세요? 반갑습니다."` ↔ `"안녕하세요? 반갑습니다!"` | 거의 동일한 문장 (구두점만 차이) |
| **0.9 이상** | `"안녕하세요? 반갑습니다."` ↔ `"안녕하세요? 만나서 반가워요."` | 같은 의미, 다른 표현 |
| **0.7-0.9** | 한국어 인사말 ↔ `"Hi, nice to meet you."` | 같은 의미, 다른 언어 (다국어 이해) |
| **0.3 이하** | 인사말 ↔ `"I like to eat apples."` | 완전히 다른 주제 |

### 실무 활용 방안

| 분야 | 구체적 활용 예시 |
|------|-----------------|
| **검색 엔진** | 정확히 일치하지 않아도 의미가 비슷한 문서 발견 |
| **대화 시스템** | 다양한 표현의 질문을 동일한 의도로 인식 |
| **문서 분류** | 유사한 주제의 문서들을 자동으로 그룹핑 |
| **번역 시스템** | 언어는 다르지만 의미가 같은 문장 매칭 |

---

## 마무리

이 튜토리얼에서 **OpenAI Embeddings**의 핵심 기능들을 학습했습니다:

### 완료된 학습 내용

- **환경 설정** - API 키와 추적 시스템 구성
- **모델 선택** - 용도에 적합한 모델 결정 방법
- **쿼리 임베딩** - 단일 텍스트의 벡터 변환
- **문서 임베딩** - 여러 문서의 일괄 처리
- **차원 조정** - 성능과 효율성의 균형점 찾기
- **유사도 계산** - 텍스트 간 의미적 유사성 측정

### 다음 학습 단계

학습한 내용을 바탕으로 다음과 같은 고급 주제로 진행할 수 있습니다:

- **RAG (Retrieval-Augmented Generation)** 시스템 구축
- **벡터 데이터베이스** 연동 및 활용
- **실제 프로덕션 환경**에서의 임베딩 시스템 구현